## Installing Avalanche with Pip

Within an Anaconda enviroment or not you can install Avalanche also with pip. In this case installing it is as simple as:

In [1]:
pip install git+https://github.com/ContinualAI/avalanche.git

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/ContinualAI/avalanche.git to /tmp/pip-req-build-vbdqwrd2
  Running command git clone -q https://github.com/ContinualAI/avalanche.git /tmp/pip-req-build-vbdqwrd2
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 1.8 MB 6.8 MB/s 
     |████████████████████████████████| 121 kB 46.4 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 418 kB 62.9 MB/s 
     |████████████████████████████████| 532 kB 57.5 MB/s 
     |████████████████████████████████| 145 kB 66.8 MB/s 
     |████████████████████████████████| 181 kB 49.3 MB/s 
     |████████████████████████████████| 63 kB 2.4 MB/s 
  Created wheel for avalanche-lib: filename=avalanche_li

In [2]:
import avalanche
avalanche.__version__

'0.1.0a0'

That's it. now we have Avalanche up and running and we can start using it! 

In [3]:
!git clone https://github.com/ContinualAI/continual-learning-baselines.git

Cloning into 'continual-learning-baselines'...
remote: Enumerating objects: 534, done.
remote: Counting objects: 100% (154/154), done.
remote: Compressing objects: 100% (93/93), done.
remote: Total 534 (delta 79), reused 123 (delta 60), pack-reused 380
Receiving objects: 100% (534/534), 104.95 KiB | 3.50 MiB/s, done.
Resolving deltas: 100% (285/285), done.


In [4]:
import sys

sys.path.insert(0, '/content/continual-learning-baselines')

In [5]:
import torch
import torchvision
from avalanche.benchmarks.datasets import MNIST
from avalanche.benchmarks.classic import SplitMNIST, PermutedMNIST

from avalanche.benchmarks.generators import nc_benchmark, ni_benchmark
from torchvision.transforms import Compose, ToTensor, Normalize, RandomCrop

In [6]:
train_transform = Compose([
    RandomCrop(28, padding=4),
    ToTensor(),
    Normalize((0.1307,), (0.3081,))
])

test_transform = Compose([
    ToTensor(),
    Normalize((0.1307,), (0.3081,))
])

mnist_train = MNIST(
    './data/mnist', train=True, download=True, transform=train_transform
)
mnist_test = MNIST(
    './data/mnist', train=False, download=True, transform=test_transform
)

  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting ./data/mnist/MNIST/raw/train-images-idx3-ubyte.gz to ./data/mnist/MNIST/raw



  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting ./data/mnist/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/mnist/MNIST/raw



  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting ./data/mnist/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/mnist/MNIST/raw



  0%|          | 0/4542 [00:00<?, ?it/s]

Extracting ./data/mnist/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/mnist/MNIST/raw



## Instance Incremental

In [7]:
scenario = ni_benchmark(
    mnist_train, mnist_test, n_experiences=10, shuffle=True, seed=1234,
    balance_experiences=True
)

train_stream = scenario.train_stream

for experience in train_stream:
    t = experience.task_label
    exp_id = experience.current_experience
    training_dataset = experience.dataset
    print('Task {} batch {} -> train'.format(t, exp_id))
    print('This batch contains', len(training_dataset), 'patterns')

Task 0 batch 0 -> train
This batch contains 5998 patterns
Task 0 batch 1 -> train
This batch contains 6000 patterns
Task 0 batch 2 -> train
This batch contains 6002 patterns
Task 0 batch 3 -> train
This batch contains 5998 patterns
Task 0 batch 4 -> train
This batch contains 6001 patterns
Task 0 batch 5 -> train
This batch contains 6001 patterns
Task 0 batch 6 -> train
This batch contains 5998 patterns
Task 0 batch 7 -> train
This batch contains 6001 patterns
Task 0 batch 8 -> train
This batch contains 5999 patterns
Task 0 batch 9 -> train
This batch contains 6002 patterns


## Class Incremental

In [8]:
scenario = nc_benchmark(
    mnist_train, mnist_test, n_experiences=10, shuffle=True, seed=1234,
    task_labels=False
)

train_stream = scenario.train_stream

for experience in train_stream:
    t = experience.task_label
    exp_id = experience.current_experience
    training_dataset = experience.dataset
    print('Task {} batch {} -> train'.format(t, exp_id))
    print('This batch contains', len(training_dataset), 'patterns')

Task 0 batch 0 -> train
This batch contains 5421 patterns
Task 0 batch 1 -> train
This batch contains 5842 patterns
Task 0 batch 2 -> train
This batch contains 5851 patterns
Task 0 batch 3 -> train
This batch contains 5958 patterns
Task 0 batch 4 -> train
This batch contains 5918 patterns
Task 0 batch 5 -> train
This batch contains 5949 patterns
Task 0 batch 6 -> train
This batch contains 6742 patterns
Task 0 batch 7 -> train
This batch contains 6265 patterns
Task 0 batch 8 -> train
This batch contains 5923 patterns
Task 0 batch 9 -> train
This batch contains 6131 patterns


## DeepSLDA Experiments

### Class incremental

In [26]:
from torch.optim import SGD
from torch.nn import CrossEntropyLoss
from torchvision import transforms
from avalanche.benchmarks.classic import SplitMNIST
from avalanche.evaluation.metrics import forgetting_metrics, accuracy_metrics, \
    loss_metrics, timing_metrics, cpu_usage_metrics, confusion_matrix_metrics, disk_usage_metrics
from avalanche.models import SimpleMLP, SLDAResNetModel
from avalanche.logging import InteractiveLogger, TextLogger, TensorboardLogger
from avalanche.training.plugins import EvaluationPlugin
from avalanche.training.supervised import Naive
from avalanche.training import StreamingLDA

from experiments.utils import set_seed, create_default_args

override_args = None

args = create_default_args({'cuda': 0, 'feature_size': 512, 'batch_size': 512,
                                'shrinkage': 1e-4, 'plastic_cov': True, 'seed': 0}, override_args)
set_seed(args.seed)
device = torch.device(f"cuda:{args.cuda}"
                          if torch.cuda.is_available() and
                          args.cuda >= 0 else "cpu")

_mu = [0.485, 0.456, 0.406]  # imagenet normalization
_std = [0.229, 0.224, 0.225]
transform = transforms.Compose([
    transforms.Grayscale(num_output_channels=3),
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=_mu,
                          std=_std)
])

scenario = SplitMNIST(n_experiences=5, train_transform=transform, eval_transform=transform)

# MODEL CREATION
model = SLDAResNetModel(device=device, arch='resnet18', imagenet_pretrained=True)
criterion = CrossEntropyLoss()

# DEFINE THE EVALUATION PLUGIN and LOGGERS
# The evaluation plugin manages the metrics computation.
# It takes as argument a list of metrics, collectes their results and returns
# them to the strategy it is attached to.

# log to Tensorboard
tb_logger = TensorboardLogger()

# log to text file
text_logger = TextLogger(open('log.txt', 'a'))

# print to stdout
interactive_logger = InteractiveLogger()

eval_plugin = EvaluationPlugin(
    accuracy_metrics(minibatch=True, epoch=True, experience=True, stream=True),
    loss_metrics(minibatch=True, epoch=True, experience=True, stream=True),
    timing_metrics(epoch=True, epoch_running=True),
    forgetting_metrics(experience=True, stream=True),
    cpu_usage_metrics(experience=True),
    confusion_matrix_metrics(num_classes=scenario.n_classes, save_image=False,
                             stream=True),
    disk_usage_metrics(minibatch=True, epoch=True, experience=True, stream=True),
    loggers=[interactive_logger, text_logger, tb_logger]
)

# CREATE THE STRATEGY INSTANCE
cl_strategy = StreamingLDA(model, criterion,
                          args.feature_size, num_classes=10,
                          eval_mb_size=args.batch_size,
                          train_mb_size=args.batch_size,
                          train_epochs=1,
                          shrinkage_param=args.shrinkage,
                          streaming_update_sigma=args.plastic_cov,
                          device=device, evaluator=eval_plugin)

# TRAINING LOOP
print('Starting experiment...')
results = []
for experience in scenario.train_stream:
    print("Start of experience: ", experience.current_experience)
    print("Current Classes: ", experience.classes_in_this_experience)

    # train returns a dictionary which contains all the metric values
    res = cl_strategy.train(experience)
    print('Training completed')

    print('Computing accuracy on the whole test set')
    # test also returns a dictionary which contains all the metric values
    results.append(cl_strategy.eval(scenario.test_stream))

Starting experiment...
Start of experience:  0
Current Classes:  [1, 4]
-- >> Start of training phase << --


/usr/local/lib/python3.7/dist-packages/avalanche/training/plugins/evaluation.py:82: UserWarning: No benchmark provided to the evaluation plugin. Metrics may be computed on inconsistent portion of streams, use at your own risk.
  "No benchmark provided to the evaluation plugin. "


100%|██████████| 25/25 [00:57<00:00,  2.28s/it]
Epoch 0 ended.
	DiskUsage_Epoch/train_phase/train_stream/Task000 = 136774.2520
	DiskUsage_MB/train_phase/train_stream/Task000 = 136774.2520
	Loss_Epoch/train_phase/train_stream/Task000 = 0.2457
	Loss_MB/train_phase/train_stream/Task000 = 0.0567
	RunningTime_Epoch/train_phase/train_stream/Task000 = 0.1062
	Time_Epoch/train_phase/train_stream/Task000 = 57.0110
	Top1_Acc_Epoch/train_phase/train_stream/Task000 = 0.9560
	Top1_Acc_MB/train_phase/train_stream/Task000 = 0.9932
-- >> End of training phase << --
Training completed
Computing accuracy on the whole test set
-- >> Start of eval phase << --
-- Starting eval on experience 0 (Task 0) from test stream --
100%|██████████| 5/5 [00:08<00:00,  1.65s/it]
> Eval on experience 0 (Task 0) from test stream ended.
	CPUUsage_Exp/eval_phase/test_stream/Task000/Exp000 = 100.1610
	DiskUsage_Exp/eval_phase/test_stream/Task000/Exp000 = 136778.8398
	Loss_Exp/eval_phase/test_stream/Task000/Exp000 = 0.0129
	

In [9]:
from tensorboard import notebook
notebook.list()

Known TensorBoard instances:
  - port 6006: logdir ./tb_data (started 0:11:55 ago; pid 21744)


In [10]:
notebook.display(port=6006, height=1000) 

Selecting TensorBoard with logdir ./tb_data (started 0:11:56 ago; port 6006, pid 21744).


### Instance incremental